<a href="https://colab.research.google.com/github/devhasibulislam/stranger-talk/blob/master/8801703310977.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Input, Dense, AveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical # Added this import
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imutils import paths
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import seaborn as sns
from google.colab import drive

drive.mount('/content/drive')

# Correct dataset path
path = "/content/drive/MyDrive/brain tumor/Brain tumor dataset"

if not os.path.exists(path):
    print("Path not found!")
else:
    print("Path OK")

# Load dataset
image_paths = list(paths.list_images(path))
images = []
labels = []

for img_path in image_paths:
    label = img_path.split(os.path.sep)[-2]
    image = cv2.imread(img_path)
    if image is not None:
        image = cv2.resize(image, (224,224))
        images.append(image)
        labels.append(label)

images = np.array(images) / 255.0
labels = np.array(labels)

# Encode labels properly (4 classes)
encoder = LabelEncoder()
labels_encoded = encoder.fit_transform(labels)
labels_encoded = to_categorical(labels_encoded)

# Train test split
train_X, test_X, train_Y, test_Y = train_test_split(
    images, labels_encoded, test_size=0.2, random_state=42
)

print("Train:", train_X.shape)
print("Test:", test_X.shape)
print("Classes:", encoder.classes_)

# Data augmentation
train_gen = ImageDataGenerator(rotation_range=15, fill_mode='nearest')

# Model setup
base = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))
x = base.output
x = AveragePooling2D(pool_size=(2,2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x)    # FIXED for 4 classes

for layer in base.layers:
    layer.trainable = False

model = Model(inputs=base.input, outputs=x)
model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Training
batch = 32
steps = len(train_X) // batch
val_steps = len(test_X) // batch

history = model.fit(
    train_gen.flow(train_X, train_Y, batch_size=batch),
    steps_per_epoch=steps,
    validation_data=(test_X, test_Y),
    validation_steps=val_steps,
    epochs=10
)

# Evaluation
pred = model.predict(test_X)
pred = np.argmax(pred, axis=1)
actual = np.argmax(test_Y, axis=1)

print(classification_report(actual, pred, target_names=encoder.classes_))

# Confusion matrix
cm = confusion_matrix(actual, pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d',
            xticklabels=encoder.classes_,
            yticklabels=encoder.classes_)
plt.title("Confusion Matrix")
plt.show()

acc = np.sum(np.diag(cm)) / np.sum(cm)
print("Final Accuracy:", acc)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Path OK
Train: (2611, 224, 224, 3)
Test: (653, 224, 224, 3)
Classes: ['glioma_tumor' 'meningioma_tumor' 'no_tumor' 'pituitary_tumor']


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 22,852,004 (87.17 MB)

 Trainable params: 1,049,220 (4.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 455s 6s/step - accuracy: 0.4944 - loss: 1.6957 - val_accuracy: 0.7213 - val_loss: 0.7064
Epoch 2/10
 1/81 ━━━━━━━━━━━━━━━━━━━━ 5:07 4s/step - accuracy: 0.6875 - loss: 0.6608

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


81/81 ━━━━━━━━━━━━━━━━━━━━ 86s 1s/step - accuracy: 0.6875 - loss: 0.6608 - val_accuracy: 0.7213 - val_loss: 0.6978
Epoch 3/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 497s 6s/step - accuracy: 0.6928 - loss: 0.7727 - val_accuracy: 0.7856 - val_loss: 0.6029
Epoch 4/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.8438 - loss: 0.4172 - val_accuracy: 0.7718 - val_loss: 0.6209
Epoch 5/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 499s 6s/step - accuracy: 0.7271 - loss: 0.6715 - val_accuracy: 0.7948 - val_loss: 0.5564
Epoch 6/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 86s 1s/step - accuracy: 0.6250 - loss: 0.7949 - val_accuracy: 0.7825 - val_loss: 0.5626
Epoch 7/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 437s 5s/step - accuracy: 0.7378 - loss: 0.6329 - val_accuracy: 0.8208 - val_loss: 0.5024
Epoch 8/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 86s 1s/step - accuracy: 0.7188 - loss: 0.6243 - val_accuracy: 0.8178 - val_loss: 0.5106
Epoch 9/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7478 - loss: 0.6288